In [2]:

# Package imports
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.linear_model
import matplotlib
import math
import tensorflow as tf

print("Started")

class NN():
    
    def __init__(self):    
                        
        self.classes = 2 #2 or 3
        self.steps_count = 2001

        self.samples_count = 201
        self.batch_size = 200
        self.batch_size = min(self.batch_size, self.samples_count-1)
    
        self.reg_lambda = 0.01
        self.learning_rate = 0.01
        
        self.input_dim = 2
        
        self.hidden_dim1 = 15
        self.hidden_dim2 = 15
        self.hidden_dim3 = 15
        
        self.activization_type = "Relu" #Sigmoid, Tanh, Relu
        
        for key, value in config.items():
            setattr(self, key, value)

        self.classes = max(min(self.classes, 3), 0)
        self.output_dim = self.classes
        
        np.random.seed(0)
                
        # hidden layer1
        w1 = np.random.randn(self.input_dim, self.hidden_dim1) / np.sqrt(self.input_dim)
        #self.W1 = tf.Variable(tf.truncated_normal([self.input_dim, self.hidden_dim1]))
        self.W1 = tf.cast(tf.Variable(w1), tf.float32)
        self.b1 = tf.Variable(tf.zeros([self.hidden_dim1]))     

        w2 = np.random.randn(self.hidden_dim1, self.hidden_dim2) / np.sqrt(self.hidden_dim1)
        #self.W2 = tf.Variable(tf.truncated_normal([self.hidden_dim1, self.hidden_dim2]))
        self.W2 = tf.cast(tf.Variable(w2), tf.float32)
        self.b2 = tf.Variable(tf.zeros([self.hidden_dim2])) 

        w3 = np.random.randn(self.hidden_dim2, self.hidden_dim3) / np.sqrt(self.hidden_dim2)
        #self.W3 = tf.Variable(tf.truncated_normal([self.hidden_dim2, self.hidden_dim3]))
        self.W3 = tf.cast(tf.Variable(w3), tf.float32)
        self.b3 = tf.Variable(tf.zeros([self.hidden_dim3]))     

        w4 = np.random.randn(self.hidden_dim3, self.output_dim) / np.sqrt(self.hidden_dim3)
        #self.W4 = tf.Variable(tf.truncated_normal([self.hidden_dim3, self.output_dim]))
        self.W4 = tf.cast(tf.Variable(w4), tf.float32)
        self.b4 = tf.Variable(tf.zeros([self.output_dim]))     

    def generateData(self):       
        
        np.random.seed(0)
        
        if (self.classes == 2):
            x, y = sklearn.datasets.make_moons(self.samples_count, noise=0.20)
        else:
            x, y = sklearn.datasets.make_blobs(self.samples_count, n_features=2, center_box=(-1.0, 1.0))
            
        self.train_x = x
        self.train_y = np.eye(np.max(y) + 1)[y]
    
    def accuracy(self, predictions, y):
        
        # Calculating the loss
        corect_logprobs = -np.log(np.amax(predictions * y, axis=1))        
        data_loss = np.sum(corect_logprobs)
                        
        # Add regulatization term to loss (optional)
        data_loss += self.reg_lambda / 2 * (np.sum(np.square(self.W1.eval())) +
                                            np.sum(np.square(self.W2.eval())) + 
                                            np.sum(np.square(self.W3.eval()))
                                 )
        size = len(y)

        return 1./size * data_loss * 100     
        
    def train(self):
    
        activization_func = tf.nn.relu
        
        x = tf.placeholder(dtype=tf.float32)
        y = tf.placeholder(dtype=tf.float32)
               
        def model(x, y):
        
            layer1 = activization_func(tf.matmul(x, self.W1) + self.b1)
            layer2 = activization_func(tf.matmul(layer1, self.W2) + self.b2)
            layer3 = activization_func(tf.matmul(layer2, self.W3) + self.b3)
            output = tf.matmul(layer3, self.W4) + self.b4
                    
            return output
        
        output = tf.nn.softmax_cross_entropy_with_logits(model(x, y), y)
        loss = tf.reduce_mean(output)
        optimizer = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(loss)
        
        test = tf.nn.softmax(model(x, y))
        
        errors = []
        with tf.Session() as session:
            tf.initialize_all_variables().run()
            
            for i in xrange(0, self.steps_count):
                
                offset = (i * self.batch_size) % (self.samples_count - self.batch_size)
                xx = self.train_x[offset:(offset + self.batch_size)]
                yy = self.train_y[offset:(offset + self.batch_size)]
                
                _, error = session.run([optimizer, loss], feed_dict={x: xx, y: yy})
                errors.append(error)
                
                if (i % 5000 == 1):
                    predictions = session.run([test], feed_dict={x: xx, y: yy})
                    print('Loss: %.1f%%' % self.accuracy(predictions[0], yy))
                
            predictions = session.run([test], feed_dict={x: self.train_x, y: self.train_y})
            print('Final Loss: %.1f%%' % self.accuracy(predictions[0], self.train_y))
            
            #print("errors ", errors)
            
            plt.plot([np.mean(errors[i-50:i]) for i in range(len(errors))])
            plt.show()
    
print("Reloaded")

config = {}

nn = NN()
nn.generateData()
nn.train()


Started
Reloaded
Loss: 81.1%
Final Loss: 25.0%


In [ ]:
print("abc")